# EE 341–Audio Fingerprint / Shazam - Lab 6

In [402]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import fftpack
import scipy.io.wavfile as wav
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage import gaussian_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion, iterate_structure
import scipy.signal as signal
from scipy.signal import argrelextrema
import simpleaudio as sa
import os
import math
from scipy.stats import mode

import scipy.io.wavfile as wav
from scipy.io import wavfile
from scipy import signal
import scipy
import scipy.io as sio
from scipy.signal import butter, lfilter, freqz, tf2zpk
import simpleaudio as sa
from IPython.display import Audio
from pydub import AudioSegment # allows conversion from mp3 to wav
import hashlib
import time

## Background

In this project, you will build a simple Shazam-like system to identify a short clip of a song from a database of songs. Shazam (supposedly) has a database of songs matching their unique features(fingerprints). When the system is fed with a song with its name to be detected, the system does the following
- Obtains the song’s fingerprint 
- Matches the fingerprint with the ones that are stored in the database
- Returns a song whose fingerprint has the closest match to the ones from the song to be detected.

The feature of a song is obtained by marking its local peaks location in the magnitude of its spectrogram. A spectrogram is a plot of Time-Dependent Fourier Transform of a song.

Spectrogram and Time-Dependent Fourier Transform

A spectrogram is an extension of a spectrum. Recall that a spectrum could be generated by Fourier Transform for a whole signal. It is a 2D graph (Magnitude / Phase vs Frequency) which shows the frequency components of the signal. Whereas the spectrogram is a 3D plot that adds a time dimension on the top of the two dimensions from Fourier Transform (Magnitude / Phase and Frequency). A spectrogram of a signal could be obtained by Time-Dependent Fourier Transform (which is also called Short-time Fourier Transform). It could be interpreted as a Fourier Transform that depends on time. Here the Fourier Transform no longer applies to the whole signal, instead, it applies to a fixed short-time segment of the signal. Thus, as we choose different segments of the signal, we will obtain different Fourier Transforms of the segments. In this way, we could plot the spectrogram of the whole signal.

So far we have learned that the database of the Shazam-like system stores a table of song names with the features corresponding to the song. For each song, we mark the frequencies and time of the peaks as features. To simplify calculation, we will only use paris of peaks that are close in both time and frequency as features. Thus, the database would keep a table with one row for each peak pair with the song id they belong to like follows

| t1| t2 | f1 | f2|songid |
|---|--- |----|---|-------|
|324|328 |  26|34 |"song1"|
|...|... |...|...|...|

For each song in the Shazam database, these feature pairs arestored in a hash table for easy access. The hash value is calculated from the vector $(f_1, f_2, t_2- t_1)$ so that the same frequencies and separation in time are considered a match. In this way, it will be robust to possible time distortion. 

We could then store the timing $t_1$ and $songid$ in the hash table. When a clip of song is to be identified, the list of pairs of peaks is produced, just as it would havebeen for a song in the database. Then the hash table is searched for each pair in the clip. This will produce a list of matches, each with different stored values of $t_1$ and songid. Some of these matches will be accidental, either because the same peak pair occurred at another time or in another song, or because the hash table had a collision. However, we expect the correct song match to have a consistent timing offset from the clip. That is, the difference between $t_1$ for the song and $t_1$ for the clip should be the same for all correct matches. The song with the most matches for a single timing offset is declared the winner.


We will use `SciPy` package for the whole lab.

## Part 1

Part 1 of the project will be buiding the database for the songs. You would use any MP3 files you want. (We suggest the MP3 with vocal since Shazam performs worse for the ones without vocal) The main steps are the following:
- Read in the song using `wavfile.read`.
- Average the two channels, subtract the mean, and downsample.
- Compute the spectrogram of the song using `spectrogram`.
- Find the local peaks of the spectrogram by using `circshift` in a loop.
- Threshold the result of step 5 to end up with a specified rate of peaks retained per second of sound.
- Find pairs of proximal peaks and add load them into a hash table.

### 1.1 Reading an MP3 File

In [302]:
# param trained: determine the file is for training database or importing to find a match
def wav_load(file_name, trained = True): 
    """
    Takes in a mp3 file and returns the signal data and sampling rate
    """
    header = file_name.rpartition('.')[0]
    dst = '/Users/shuku/Desktop/EE 341/Lab6/wavSongList/'+header+'.wav'
    # convert wav to mp3
    if(trained):
        sound = AudioSegment.from_mp3('/Users/shuku/Desktop/EE 341/Lab6/songlist/'+file_name)
    else:
        sound = AudioSegment.from_mp3(file_name)
        
    sound.export(dst, format="wav")
    sr, data = wav.read(dst) 
    
    return sr, data 
   

### 1.2 Preprocessing

Here you should average the two channels of the audio file, subtract the mean of the signal, and downsample it to 8000 Hz using `signal.resample`.

In [304]:
def preprocess(data, sr) :
    """
    Takes in signal data and sampling rate, averages the two channels when necessary, subtracts
    the mean of the signal, and downsamples it to 8000 Hz
    """
    sec = len(data) / sr
    num = int(sec * 8000) # number of samples

    channel1 = data[:, 0]
    channel2 = data[:, 1]

    avg = (channel1 + channel2) / 2
    mean = np.mean(data)

    substraction = avg - mean

    resampled = signal.resample(substraction, num)

    return resampled

### 1.3 Creating Spectrogram

Here you should construct the spectrogram of the signal using `signal.spectrogram`

In [305]:
def specgram(dat, sr = 8000) :
    """
    Takes in signal data and its sampling rate and produces a spectrogram for the signal
    f=Array of sample frequencies.
    t=Array of segment times.
    sxx=Spectrogram of x. By default, the last axis of Sxx corresponds to the segment times.
    """
    
    f, t, sxx = signal.spectrogram(dat, sr)
    
    plt.pcolormesh(t, f, sxx)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()
    return f, t, sxx

### 1.4 Finding Local Peaks

Next, we find the local peaks of the spectrogram and produce a binary matrix (the same size as the spec- trogram) with a 1 at each location of a peak.

Create a function `peak_counts` that does the following:
- `peak_counts`: takes as input `peak_matrix` and checks if entries in this matrix (which has binary values) have a 1. If so increase a counter and then return the count of the number of peaks.



In [306]:
def peak_counts(peak_matrix) :
    """
    Takes in a binary matrix denoting the peaks of a spectrogram
    and eturns the total bumber of peaks in the matrix
    """
    peak_counts = 0
    for i in range(len(peak_matrix)) :
        for j in range(len(peak_matrix[0])) :
            if (peak_matrix[i,j] == 1) :
                peak_counts = peak_counts + 1
    return peak_counts

### 1.5 Thresholding

We want to only use the larger peaks.Set up a poper threshold and get rid of the small peaks.

For example,  if $A$ is your matrix consider checking when the differences of two consecutive row elements of the matrix are larger than 2xmean. For instance,
$$A[i,j] - A[i - 1,j] - 0.2 \mu > 0$$ and
 $$A[i,j] - A[i + 1,j] - 0.2  \mu > 0$$
 where $\mu$ is the mean.

In [307]:
def find_peaks(f, t, sxx) : 
    """
    Takes in spectrogram data sxx and returns a binary matrix
    denoting significant peaks in the data whose magnitude
    surpasses a variable threshold
    """
    peak_matrix = np.zeros((len(f), len(t)))
    mean = np.mean(sxx)
    for i in range(len(f) - 1) :
        for j in range(len(t)) :
            if (sxx[i][j] - sxx[i - 1][j] - 0.8 * mean > 0) & (sxx[i][j] - sxx[i + 1][j] - 0.8 * mean > 0) :
                peak_matrix[i][j] = 1
    print('number of peaks: ', peak_counts(peak_matrix))

    plt.figure()
    plt.subplot(2, 1, 1)
    plt.pcolormesh(t, f, sxx)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()
    plt.subplot(2, 1, 2)
    plt.imshow(peak_matrix);
    plt.colorbar()
    plt.show()
    
    return peak_matrix

## Part 1.6 and 1.7


### 1.6 Finding peak pairs

Define a function `ctp` that does the following. The code finds pairs by considering each peak and looking for other peaks within a designated window located relative to it. During the search, we limit the number of pairs that we accept by the parameter fanout. The code is written to scan through the window column by column, accepting the first pairings that it finds. 

### 1.7 
- `simple_hash`:  Simple hash function which hashes input frequency data. Given the frequencies and the time difference of the peak pair (f_1, f_2, t_2 − t_1) compute the hash value. For example: hashvalue = $mod(round(size\*1000000\*(log(abs(f1)+2) + 2\*log(abs(f2)+2) + 3\*log(abs(deltaT)+2)) ), size) + 1. You will use this in the next function
- `add_to_table`: Adds peak-pair information from song's fingerprint to the hashtabl. In this function, you first compute the hashvalue using 'simple_hash', after that check if the hashvalue if used. If it is not used add the songid and frequencies and corresponding time to the hastable, otherwise append the songid and frequencies and so on.
- `fingerprint`: Create a function that takes in an array of data and its sample rate, and returns a peak matrix of the data
    It should combine functions from section one into one place and removes graphing commands
- `make_database`: searches for all MP3 files in a designated folder and processes them if they are not already in the database

    Create a function that changes global variables: `hashtable`, `songid` by
    checking to see if the database (hashtable and songid files) already has been created in local directory. 
    If so, then these are imported. If not, then they are created.
    Then it checks if all files in the "songs" folder of local directory have been added to the database.
    If not then it adds them.
    Saves database (hashtable and songid files) in local director



In [308]:
def ctp(peaks):
    """
    Finds pairs of peaks that are close in time and frequency from
    a binary matrix that denotes the location of peaks in discrete time
    and frequency.
    Returns an array of tuples containing peak pair data
    """
    
    del_t = 35
    del_f = 30
    fanout = 3 # maiximum distance between peaks
    
    f, t = np.nonzero(peaks)
    tup = np.zeros((fanout*len(f), 4))
    
    index = 0
    
    for i in range(len(f)):
        links = 0
        for f2 in range(min(peaks.shape[0], f[i] + 1), min(peaks.shape[0], f[i] + del_f)):
            if (peaks[f2, t[i]]):
                tup[index,:] = [t[i], t[i], f[i], f2]
                links = links + 1
                index = index + 1
            if links >= fanout:
                break
                    
        for t2 in range(min(peaks.shape[0], t[i] + 1), min(peaks.shape[1], t[i] + del_t)):
            if (links >= fanout):
                break
            for f2 in range(max(1, f[i] - del_f), min(peaks.shape[0], f[i] + del_f)):
                if (links >= fanout):
                    break
                if (peaks[f2, t2]):
                    tup[index,:] = [t[i], t2, f[i], f2]
                    links = links + 1
                    index = index + 1
    tup = tup[0:index,:]
    
    return tup # return the peak pair data

In [309]:
def fingerprint(sr, data):
    """
    Takes in an array of data and its sample rate. Returns a peak matrix of the data
    Combines functions from section one into one place and removes graphing commands
    """
    #TODO preprocess the data and return the peak matrix
    data = preprocess(data, sr)
    fs = 8000.0

    f, t, sxx = signal.spectrogram(data, fs)
    
    peak_matrix = np.zeros((len(f), len(t)))
    mean = np.mean(sxx)
    for i in range(len(f) - 1) :
        for j in range(len(t)) :
            if (sxx[i][j] - sxx[i - 1][j] - 0.8 * mean > 0) & (sxx[i][j] - sxx[i + 1][j] - 0.8 * mean > 0) :
                peak_matrix[i][j] = 1
            
    return peak_matrix

In [384]:
def simple_hash(f1, f2, deltaT, size):
    """
    Simple hash function which hashes input frequency data
    f1: the 
    """
    return np.mod(round(size*1000000*(np.log(abs(f1)+2) + 2*np.log(abs(f2)+2) + 3*np.log(abs(deltaT)+2)) ), size) + 1

def add_to_table(tup, songid):
    """
    This function changes global variable: hashtable
    Adds peak-pair information from song's fingerprint to the hashtable
    """
    global hashtable

    hash_song_dict = hashtable[0]
    hash_pair_dict = hashtable[1]



    for m in range(len(tup)):
        # s_hash is the key of the two dict
        s_hash = simple_hash(tup[m][2], tup[m][3], tup[m][1]-tup[m][0], 100000)
        
        value = [tup[m][0], tup[m][1], tup[m][2], tup[m][3], songid]
        

        if (s_hash not in hash_song_dict.keys()): 
            # TODO
            hash_song_dict[s_hash] = []
            hash_pair_dict[s_hash] = []
        # append value to the corresponding keys
        hash_song_dict[s_hash].append(songid) 
        hash_pair_dict[s_hash].append(value)
        
    
def make_database():
    """
    This functino changes global variables: hashtable, songid
    Checks to see if database (hashtable and songid files) have already been created in local directory
    If so, then these are imported. If not, then they are created.
    Checks if all files in the "songs" folder of local directory have been added to the database.
    If not then it adds them.
    Saves database (hashtable and songid files) in local directory
    """
    global hashtable
    global songid 
    
    #TODO
    
    #Checks if local copy of database exists and imports
    if (os.path.exists('hashtable.npy') and os.path.exists('songid.npy')):
        #TODO
        print('exists!')
        hashtable = np.load('hashtable.npy', allow_pickle=True)
        songid = np.load('songid.npy', allow_pickle=True)
    else: #Creates one if not
        #TODO
        # store as dictionary
        print('not exists!')
        hash_song_dict = {} # songid
        hash_pair_dict = {} # feature
        
#       hashtable: list of 2 dict
        hashtable = [hash_song_dict, hash_pair_dict]

        np.save('hashtable.npy', hashtable)
        
        songid = np.empty(10).astype(str)
        np.save('songid.npy', songid)

    path = '/Users/shuku/Desktop/EE 341/Lab6/songlist'
    songlist = os.listdir(path)
    # this part not sure
    for i in range(len(songlist)):
        song_name = songlist[i]
        if(song_name[-4:] == '.mp3'): # make sure song in music file
            if (song_name not in songid):
                print(song_name)
                songid[i] = song_name # add to songid.npy
                sr, data = wav_load(song_name)
                peak_matrix = fingerprint(sr, data)
                tup = ctp(peak_matrix)
                add_to_table(tup, song_name) # add to table
    np.save('hashtable.npy', hashtable)
    np.save('songid.npy', songid)

In [427]:
T1 =time.time()
make_database()
T2 = time.time()
print('Program running time (in second)', T2 - T1)

exists!
Canon.mp3
Men Without Hats.mp3
Blinding Lights.mp3
Imagine.mp3
VIva La Vida.mp3
Program running time (in second) 200.9102168083191


In [428]:
print(songid)

['Five Hundred Miles.mp3' '6.7903865365e-313' 'Canon.mp3'
 'Men Without Hats.mp3' 'Blinding Lights.mp3' 'Imagine.mp3'
 'VIva La Vida.mp3' '2.440295160183e-312' '2.31297541238e-312'
 '1.082217853946e-312']


In [330]:
print(hashtable[0])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Part 2

Part 2 of the project will be buiding the part of Shazam that identifies the segment of music.
The main steps of the algorithm are the following:
- Load HASHTABLE and SONGID that were createdin part 1. 
- Prepare a clip of music for identification.
- Extract the list of frequency pairs from the clip.
- Look up matches in the hash table, calculate time offsets, and sort them by song. 
- Identify the song with the most matches for a single consistent timing offset.

### 2.1 and 2.2 Extracting Features from the Input Clip and Recovering matches from hash table

Use the frequencies and the time difference of the peak pair (f1, f2, t2 − t1) as inputs to the hash function. Then extract the two lists from the hash table saved at the location provided by the hash function. Then, convert the timing list to a list of timing offsets by subtracting the time t1 that the peak pair occurred in the clip. This list of offsets is what we will save.

In [398]:
def match_segment(sr, data):
    """
    Takes in the data and sample rate of an audio file
    Creates a fingerprint for the data and attempts to match this against the database
    Fingerprint is then checked for collisions against the database hashtable
    Determines a confidence level based on the percentage of occurences for the most common timing offset
    Returns the songid of the most likely match
    Returns the confidence in the match
    """
    global hashtable
    global songid
    
    hashtable = np.load('hashtable.npy', allow_pickle=True)
    songid = np.load('songid.npy', allow_pickle=True)
    hash_song_dict = hashtable[0]
    hash_pair_dict = hashtable[1]
    
    hashTableSize = len(hashtable[0])
    
    fp = fingerprint(sr, data)
    tup = ctp(fp)
    
    mat = {}
    
        
    for k in range(len(tup)):
        clip_hash = simple_hash(tup[k][2], tup[k][3], tup[k][1]-tup[k][0], 100000)# TODO compute the hash value
        if (clip_hash in hash_song_dict.keys()):
            matchID = hash_song_dict[clip_hash] # songid, a list
            matchTime = hash_pair_dict[clip_hash] # those features, a list
            
            for j in range(len(matchID)):
                song_name = matchID[j]
                if song_name in mat:
                    mat[song_name] += 1
                else:
                    mat[song_name] = 1
    
    return mat

### 2.3 Identifying the song

Find the song that has the most occurrences of any single timing offset. This is most easily done by looping through each song and using the mode command. 

In [429]:
# example usage
T3 = time.time()
sr1, data1 = wav_load("viva.mp3", trained=False) #Not in database
songs = match_segment(sr1, data1)
T4 = time.time()
print('program run time:', (T4 - T3))
# load the files given to you

program run time: 9.155421257019043


In [430]:
import operator
sorted_d = dict( sorted(songs.items(), key=operator.itemgetter(1),reverse=True))
print('Best Match Song in descending order by value : ',sorted_d)

Best Match Song in descending order by value :  {'VIva La Vida.mp3': 1642878, 'Blinding Lights.mp3': 1233446, 'Men Without Hats.mp3': 774946, 'Imagine.mp3': 720134, 'Canon.mp3': 382694, 'Five Hundred Miles.mp3': 234464}


In [431]:
# Test
T3 = time.time()
sr1, data1 = wav_load("Blinding Lights (sample).mp3", trained=False) #Not in database
songs = match_segment(sr1, data1)
T4 = time.time()
print('program run time:', (T4 - T3))
sorted_d = dict(sorted(songs.items(), key=operator.itemgetter(1),reverse=True))
print('Best Match Song in descending order by value : ',sorted_d)

program run time: 13.609328031539917
Best Match Song in descending order by value :  {'Blinding Lights.mp3': 7711840, 'VIva La Vida.mp3': 7215528, 'Imagine.mp3': 5562532, 'Men Without Hats.mp3': 5288820, 'Canon.mp3': 2924328, 'Five Hundred Miles.mp3': 2636995}


In [432]:
T3 = time.time()
sr1, data1 = wav_load("Canon (mp3cut.net).mp3", trained=False) #Not in database
songs = match_segment(sr1, data1)
T4 = time.time()
print('program run time:', (T4 - T3))
sorted_d = dict(sorted(songs.items(), key=operator.itemgetter(1),reverse=True))
print('Best Match Song in descending order by value : ',sorted_d)

program run time: 11.605187177658081
Best Match Song in descending order by value :  {'Canon.mp3': 4609537, 'Imagine.mp3': 3787366, 'Blinding Lights.mp3': 3745250, 'VIva La Vida.mp3': 3482110, 'Men Without Hats.mp3': 3012780, 'Five Hundred Miles.mp3': 2986689}
